# Regresión logística aplicada a iris

**Lectura del corpus y partición:**

In [ ]:
# Importar la biblioteca NumPy para cálculos numéricos
import numpy as np

# Importar la función para cargar el conjunto de datos Iris desde scikit-learn
from sklearn.datasets import load_iris

# Importar la función para dividir datos en conjuntos de entrenamiento y prueba
from sklearn.model_selection import train_test_split

# Cargar el conjunto de datos Iris
iris = load_iris()

# Extraer las características y convertirlas al tipo de dato float16 para optimizar el uso de memoria
X = iris.data.astype(np.float16)

# Extraer las etiquetas y convertirlas al tipo de dato entero sin signo (uint)
y = iris.target.astype(np.uint)

# Dividir los datos en conjuntos de entrenamiento y prueba, con un 20% de los datos como prueba
# 'shuffle=True' asegura que los datos se barajen antes de dividirse
# 'random_state=23' asegura que la división sea reproducible al usar una semilla específica
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)


**LogisticRegression:** $\;$ implementación de regresión logística en sklearn

In [ ]:
# Importar la clase LogisticRegression de scikit-learn para realizar regresión logística
from sklearn.linear_model import LogisticRegression

# Importar la función accuracy_score para calcular la precisión de las predicciones
from sklearn.metrics import accuracy_score

# Crear una instancia del clasificador de regresión logística con una semilla aleatoria para reproducibilidad
clf = LogisticRegression(random_state=23).fit(X_train, y_train)

# Predecir las etiquetas para el conjunto de datos de prueba
y_test_pred = clf.predict(X_test)

# Calcular el error en el conjunto de prueba como 1 menos la precisión
err_test = 1 - accuracy_score(y_test, y_test_pred)

# Imprimir el error de prueba formateado como un porcentaje con un decimal
print(f"Error de test: {err_test:.1%}")


**Warnings:** $\;$ sklearn es un poco "insistente" con los warnings; ignoraremos los avisos sobre convergencia

In [ ]:
# Importar el módulo warnings para controlar las advertencias
import warnings

# Importar ConvergenceWarning de sklearn.exceptions para capturar advertencias específicas de convergencia
from sklearn.exceptions import ConvergenceWarning

# Configurar el módulo warnings para ignorar todas las advertencias de la categoría ConvergenceWarning que provienen de la biblioteca sklearn
warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn")


**Solvers:** $\;$ el parámetro `solver` de LogisticRegression permite elegir entre diferentes solvers (algoritmos de optimitzación)

In [ ]:
# Definir una lista de solvers a probar con el modelo de regresión logística
for solver in ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']:
    # Crear y entrenar el modelo de regresión logística con cada solver
    # 'max_iter=10000' establece un número alto de iteraciones para permitir la convergencia
    clf = LogisticRegression(random_state=23, solver=solver, max_iter=10000).fit(X_train, y_train)

    # Predecir las etiquetas para el conjunto de prueba y calcular el error
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))

    # Imprimir el error de prueba para cada solver
    print(f"Error de test después de entrenar con el solver {solver!s}: {err_test:.1%}")


**Tolerancia:** $\;$ el parámetro `tol` establece un umbral de tolerancia para acabar el entrenamiento (1e4 por defecto)

In [ ]:

# Iterar sobre una lista de diferentes valores de tolerancia para el criterio de parada
for tol in (1e-4, 1e-2, 1, 1e2, 1e4):
    # Crear y entrenar el modelo de regresión logística con cada valor de tolerancia
    # 'tol' define el criterio de parada para la optimización: el proceso se detiene cuando el error es menor que 'tol'
    clf = LogisticRegression(tol=tol, random_state=23, max_iter=10000).fit(X_train, y_train)

    # Predecir las etiquetas para el conjunto de prueba y calcular el error
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))

    # Imprimir el error de prueba para cada valor de tolerancia
    print(f"Error de test con tolerancia {tol}: {err_test:.1%}")

# Los valores de 'tol' van desde muy pequeños (1e-4) hasta muy grandes (1e4), lo que permite evaluar cómo la precisión del modelo varía con diferentes criterios de parada en la optimización.


**Regularización:** $\;$ el parámetro `C` (positivo, $1.0$ por defecto) des-regulariza el criterio de entrenamiento
* **Posibilidad de subajuste:** $\;$ con un valor próximo a cero (máxima regularización)
* **Posibilidad de sobreajuste:** $\;$ con un valor positivo muy alto (mínima regularización)

In [ ]:
for C in (1e-2, 1e-1, 1, 1e1, 1e2):
    clf = LogisticRegression(C=C, random_state=23, max_iter=10000).fit(X_train, y_train)
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))
    print(f"Error de test con C {C:g}: {err_test:.1%}")

**Early stopping:** $\;$ ahorramos cálculo y evitamos sobre-entrenamiento ("regularizamos") acabando pronto (en pocas iteraciones)

In [ ]:
# Este código evalúa cómo la limitación en el número máximo de iteraciones afecta el rendimiento del modelo.
# Valores bajos de 'max_iter' pueden llevar a una convergencia prematura, posiblemente resultando en un modelo subóptimo.

# Iterar sobre una lista de diferentes valores para el número máximo de iteraciones
for max_iter in (10, 20, 50, 100):
    # Crear y entrenar el modelo de regresión logística con cada valor de max_iter
    # 'max_iter' define el número máximo de iteraciones del proceso de optimización
    clf = LogisticRegression(random_state=23, max_iter=max_iter).fit(X_train, y_train)

    # Predecir las etiquetas para el conjunto de prueba y calcular el error
    err_test = 1 - accuracy_score(y_test, clf.predict(X_test))

    # Imprimir el error de prueba para cada valor de max_iter
    print(f"Error de test con max_iter {max_iter}: {err_test:.1%}")




**Lo ponemos todo junto**
---

Mostrar los resultados de error de test (err_test) para combinaciones de diferentes valores de:

1. `max_iter` (probar valores 10, 50, 100, 200, 500, 800, 1000)
2. `solver` (utilizar sag, saga, lbfgs y netwon-cg que son los solvers para problemas multiclase
3. `C` (regularización: 0.01, 0.1, 1.0 , 10.0, 100)
4. `tol` (tolerancia: probar valores 10e-4, 10e-2, 1.0, 10e2, 10e4)



In [ ]:
# Definir los parámetros a probar
max_iters = [10, 50, 100, 200, 500, 800, 1000]
solvers = ['sag', 'saga', 'lbfgs', 'newton-cg']
Cs = [0.01, 0.1, 1.0, 10.0, 100]
tols = [1e-4, 1e-2, 1, 1e2, 1e4]

print('   solver     tol       C max_iter  ete')
print('--------- ------- ------- -------- -----')

# Iterar sobre las combinaciones de max_iter, solver, C y tol
for max_iter in max_iters:
    for solver in solvers:
        for C in Cs:
            for tol in tols:
                # Crear y entrenar el modelo
                clf = LogisticRegression(max_iter=max_iter, solver=solver, C=C, tol=tol, random_state=23).fit(X_train, y_train)
                # Calcular el error de prueba
                ete = 1 - accuracy_score(y_test, clf.predict(X_test))
                # Imprimir los resultados
                print(f'{solver:>9} {tol:.1e} {C:.1e} {max_iter:8d} {ete:5.1%}')


**Pregunta**:
Indica cuál crees que es la mejor combinación de iteraciones, solver, C y tolerancia